In [1]:
# Import libraries
'''
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
'''

# Pretty display for notebooks
'''
%matplotlib inline
'''

# Allows the use of display() for DataFrames
from IPython.display import display 

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
weather = pd.read_csv("../asset/weather.csv")
key = pd.read_csv("../asset/key.csv")
submission_example = pd.read_csv("../asset/sampleSubmission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("weather  data : ", weather.shape)
print("key  data : ", key.shape)
print("submission  data : ", submission_example.shape)

import pickle
import costum_functions as cf

slack_url = pickle.load(open('slackUrl.pickle', 'rb'))
slack_url

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_weather = weather.copy()
df_key = key.copy()
df_test = test.copy()

Train data :  (4617600, 4)
Test  data :  (526917, 3)
weather  data :  (20517, 20)
key  data :  (45, 2)
submission  data :  (526917, 2)


In [3]:
import datetime

def addDays(date):
    temp = date.split("-")
    year, month, date = int(temp[0]), int(temp[1]), int(temp[2])
    return datetime.datetime(year, month, date).weekday()

In [ ]:
# df_train_with_weather["day"] = df_train_with_weather["date"].apply(addDays)

In [4]:
df_train.tail()

,date,store_nbr,item_nbr,units
4617595,2014-10-31,45,107,0
4617596,2014-10-31,45,108,0
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [7]:
df_key.tail()

,store_nbr,station_nbr
40,41,12
41,42,14
42,43,11
43,44,12
44,45,16


In [72]:
df_weather

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,M,36,40,23,0,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,0716,1626,RA,0.0,0.07,28.82,29.91,9.1,23,11.3
2,3,2012-01-01,55,34,45,9,24,36,20,0,0735,1720,,0.0,0.00,29.77,30.47,9.9,31,10.0
3,4,2012-01-01,63,47,55,4,28,43,10,0,0728,1742,,0.0,0.00,29.79,30.48,8.0,35,8.2
4,6,2012-01-01,63,34,49,0,31,43,16,0,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
5,7,2012-01-01,50,33,42,M,26,35,23,0,-,-,,0.0,0.00,29.15,30.54,10.3,32,10.2
6,8,2012-01-01,66,45,M,M,34,46,M,M,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9
7,9,2012-01-01,34,19,27,M,17,23,38,0,-,-,UP,M,T,29.34,30.09,22.8,30,22.5
8,10,2012-01-01,73,53,63,M,55,58,2,0,0723,1738,FG+ FG BR,M,0.00,30.16,30.19,5.1,24,5.5
9,11,2012-01-01,72,48,60,7,54,56,5,0,0724,1737,FG+ FG BR,0.0,0.00,30.15,30.18,4.6,23,4.8


In [9]:
df_test.tail()

,date,store_nbr,item_nbr
526912,2014-10-26,45,107
526913,2014-10-26,45,108
526914,2014-10-26,45,109
526915,2014-10-26,45,110
526916,2014-10-26,45,111


In [12]:
df_train_key = df_train.merge(df_key, on=["store_nbr"], how="inner")
df_train_key.tail()

,date,store_nbr,item_nbr,units,station_nbr
4617595,2014-10-31,35,107,0,5
4617596,2014-10-31,35,108,0,5
4617597,2014-10-31,35,109,0,5
4617598,2014-10-31,35,110,0,5
4617599,2014-10-31,35,111,0,5


In [14]:
total = df_train_key.merge(df_weather, on=["date","station_nbr"], how="inner")
total.tail()

,date,store_nbr,item_nbr,units,station_nbr,tmax,tmin,tavg,depart,dewpoint,...,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
4617595,2014-10-31,35,107,0,5,M,M,M,M,37,...,0616,1640,,M,M,M,30.01,10.8,05,M
4617596,2014-10-31,35,108,0,5,M,M,M,M,37,...,0616,1640,,M,M,M,30.01,10.8,05,M
4617597,2014-10-31,35,109,0,5,M,M,M,M,37,...,0616,1640,,M,M,M,30.01,10.8,05,M
4617598,2014-10-31,35,110,0,5,M,M,M,M,37,...,0616,1640,,M,M,M,30.01,10.8,05,M
4617599,2014-10-31,35,111,0,5,M,M,M,M,37,...,0616,1640,,M,M,M,30.01,10.8,05,M


In [16]:
total_nonzero = total[total["units"]!=0]
total_nonzero.tail()

,date,store_nbr,item_nbr,units,station_nbr,tmax,tmin,tavg,depart,dewpoint,...,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
4617332,2014-10-29,35,66,14,5,M,M,M,M,55,...,0613,1643,,M,M,M,29.87,8.2,23,M
4617393,2014-10-30,35,16,16,5,M,M,M,M,38,...,0615,1641,,M,M,M,29.95,8.3,30,M
4617443,2014-10-30,35,66,6,5,M,M,M,M,38,...,0615,1641,,M,M,M,29.95,8.3,30,M
4617504,2014-10-31,35,16,18,5,M,M,M,M,37,...,0616,1640,,M,M,M,30.01,10.8,05,M
4617554,2014-10-31,35,66,10,5,M,M,M,M,37,...,0616,1640,,M,M,M,30.01,10.8,05,M


In [19]:
len(total_nonzero)

118696

In [ ]:
# station별로 missing data 일단 삭제해서 pairplot으로 상관관계를 보자(station 6~10)

In [64]:
# M value row 모두 삭제
# T value 모두 0으로 변환
df_station_6 = total[total["station_nbr"]==6]
df_station_7 = total[total["station_nbr"]==7]
df_station_8 = total[total["station_nbr"]==8]
df_station_9 = total[total["station_nbr"]==9]
df_station_10 = total[total["station_nbr"]==10]

In [65]:
len(df_station_6)

205572

In [ ]:
df.loc[df['column_name'] == "M"]

In [71]:
total[:100000].to_csv("total10000.csv")

In [66]:
df_station_6 = df_station_6[df_station_6.tmax != "M"]
len(df_station_6)

205128

In [68]:
df_station_6 = df_station_6[df_station_6.tavg != "M"]
len(df_station_6)

204240

In [62]:
# T value 모두 0으로 변환

def changeFloatType(a):
    a = str(a).strip()
    if a.strip() != "T"
        return float(a)
    else:
        return 0

,date,store_nbr,item_nbr,units,station_nbr,tmax,tmin,tavg,depart,dewpoint,...,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
1397379,2012-01-01,7,1,0,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
1397380,2012-01-01,7,2,0,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
1397381,2012-01-01,7,3,0,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
1397382,2012-01-01,7,4,0,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
1397383,2012-01-01,7,5,137,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
1397384,2012-01-01,7,6,0,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
1397385,2012-01-01,7,7,2,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
1397386,2012-01-01,7,8,0,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
1397387,2012-01-01,7,9,0,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
1397388,2012-01-01,7,10,0,6,63,34,49,0,31,...,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8


In [ ]:
def changeType(a):
    if(a != "M"):
        return int(a)
    else:
        return 0

def changeFloatType(a):
    a = str(a).strip()
    if(a != "M" and a.strip() != "T"):
        return float(a)
    else:
        return 0

# df["tavg"] = df["tavg"].apply(changeType)
# df["units"] = df["units"].apply(changeType)
# df["heat"] = df["heat"].apply(changeType)
# df["cool"] = df["cool"].apply(changeFloatType)
# df["depart"] = df["depart"].apply(changeType)